## Create a dataset of images and labels

In [1]:
# Read ground truth data (mat file)
import scipy
import os

data_folder = "AMC_demo_data"
gt_data_path = os.path.join(data_folder, "ground_truth.mat")
gt_data_mat = scipy.io.loadmat(gt_data_path)
gt_data = gt_data_mat["gt_data"]

In [2]:
# Separate the the gt_data by timestamps
gt_data_timestamps = []

timestamps = gt_data.shape[2]
for t in range(timestamps):
    gt_data_at_t = gt_data[:, :, t]
    gt_data_timestamps.append(gt_data_at_t)

In [3]:
# Clean unrecognized objects (remove the entries whose axies 1 are all zeros)
import numpy as np

for t in range(len(gt_data_timestamps)):
    gt_data_at_t = gt_data_timestamps[t]
    mask = ~(gt_data_at_t == 0).all(axis=1)
    gt_data_at_t = gt_data_at_t[mask]
    # If no traffic sign detected, make the gt_data_at_t empty (reserve background samples)
    if len(gt_data_at_t) == 0:
        gt_data_at_t = []
    gt_data_timestamps[t] = gt_data_at_t

In [4]:
numid_to_classid = {
    95: 0,
    96: 1,
    98: 2,
    99: 3,
    100: 2,
    101: 1,
    102: 3,
    103: 2,
    104: 2,
    105: 3, 
    106: 1,
    107: 1
}    

In [ ]:
# Parse the Prescan gt data into YOLO gt format and save into "labels" folder
IMAGE_WIDTH = 800
IMAGE_HEIGHT = 600

from utils_own import *
import os

import shutil

label_dir = os.path.join(data_folder, "labels")
if os.path.exists(label_dir):
    shutil.rmtree(label_dir)
os.makedirs(label_dir, exist_ok=True)

for t in range(len(gt_data_timestamps)):
    gt_data_at_t = gt_data_timestamps[t]
    with open(label_dir + f"\{t:05}.txt", "a") as file:
        # Separate rows
        for row in gt_data_at_t:
            yolo_gt = convert_prescan_gt_to_yolo_gt(row, IMAGE_WIDTH, IMAGE_HEIGHT, numid_to_classid, gtsdb_format=False)
            file.write(yolo_gt)

In [ ]:
# # Sanity check of generated "labels"
# import cv2

# image_width = 800
# image_height = 600

# image = cv2.imread(r"D:\HJY\graduation_perception\AMC_demo_data\CameraSensor_1\CameraSensor_1_00015.png")
# label_path = r"D:\HJY\graduation_perception\AMC_demo_data\labels\00015.txt"
# # image = cv2.imread(r"D:\HJY\graduation_perception\TSR_1_data\CameraSensor_1\00458.png")
# # label_path = r"D:\HJY\graduation_perception\TSR_1_data\labels\00458.txt"

# with open(label_path, "r") as f:
#     for row in f:
#         _, x, y, w, h = map(float, row.split())
#         # Conver xywh to pixel values
#         x = x * image_width
#         y = y * image_height
#         w = w * image_width
#         h = h * image_height

#         # y = image_height - y
#         # Convert xywh to ltrb
#         left = int(x  - w / 2)
#         top = int(y - h / 2)
#         right = int(x + w / 2)
#         bottom = int(y + h / 2)
#         # bottom = int(top + (bottom - top) * (0.6 / (0.6 + 1.6)))

#         print(left, top, right, bottom)
#         # Draw the bounding box (color: green, thickness: 2)
#         cv2.rectangle(image, (left, bottom), (right, top), (0, 255, 0), 2)

# # Show the image with bounding box
# cv2.imshow("Image with Bounding Box", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


439 259 468 364
414 281 428 330


In [3]:
# Change image file names to the same as the label file names
import os

img_folder = os.path.join(data_folder, "CameraSensor_1")
label_folder = os.path.join(data_folder, "labels")
label_file_names = os.listdir(label_folder)
img_file_names = os.listdir(img_folder)

# Delete the non-image file in the image folder
for file_name in img_file_names:
    if not file_name.endswith(".png"):
        file_to_delete = os.path.join(img_folder, file_name)
        os.remove(file_to_delete)
        break

img_file_names = os.listdir(img_folder)
# Delete the last image if the images have one more extra than the labels
if len(img_file_names) > len(label_file_names):
    img_to_delete = img_file_names[-1]
    img_to_delete_path = os.path.join(img_folder, img_to_delete)
    os.remove(img_to_delete_path)

for ind, img_file_name in enumerate(os.listdir(img_folder)):
    img_path = os.path.join(img_folder, img_file_name)

    # Get the corresponding label file name
    label_file_name = label_file_names[ind]

    # Make the new img file name (replace .txt with .png)
    new_file_name = label_file_name.replace(".txt", ".png")
    new_img_path = os.path.join(img_folder, new_file_name)

    # Rename the image file
    os.rename(img_path, new_img_path)


## Create a new dataset using frame skipping

In [12]:
data_folder = "AMC_demo_data"

In [13]:
# Frame skipping to create a new dataset
import shutil

num_frame_skip = 3

new_dataset_dir = data_folder + "_new_dataset"
new_image_folder = os.path.join(new_dataset_dir, "images")
new_label_folder = os.path.join(new_dataset_dir, "labels")
os.makedirs(new_dataset_dir, exist_ok=True)
os.makedirs(new_image_folder, exist_ok=True)
os.makedirs(new_label_folder, exist_ok=True)

# Get the original image and label file names
img_file_names = os.listdir(img_folder)
label_file_names = os.listdir(label_folder)

ind = 0
while ind < len(img_file_names):
    # Get the original image and label path
    img_path = os.path.join(img_folder, img_file_names[ind])
    label_path = os.path.join(label_folder, label_file_names[ind])

    # Copy the image and label file to the new folders
    new_img_path = os.path.join(new_image_folder, img_file_names[ind])
    new_label_path = os.path.join(new_label_folder, label_file_names[ind])
    shutil.copy(img_path, new_img_path)
    shutil.copy(label_path, new_label_path)

    ind += num_frame_skip